# Bigbasket Analysis

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../data/BigBasket_Products.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

## Step 1 :- Data Cleaning

In [ ]:
#  1. Drop Redundant Columns
# 'index' column is just row number; can be dropped
df.drop(columns = ['index'], inplace= True)

In [ ]:
# 2. Rename Columns for Consistency
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
df.columns

In [ ]:
# 3. Check for Missing Values
df.isnull().sum()

In [ ]:
# 4. Handle Missing Data
# Drop rows where product or brand is missing
df.dropna(subset=['product','brand'], inplace = True)

# Fill missing ratings with the median
df['rating'].fillna(df['rating'].median(), inplace = True)

# Fill missing descriptions with a placeholder
df['description'].fillna('No description provided.', inplace = True)

In [ ]:
# 5. Add Discount and Discount % Columns
df['discount'] = df['market_price'] - df['sale_price']
df['discount_percent'] = ((df['discount'] / df['market_price']) * 100).round(2)

In [ ]:
# 6. Add Price Range Buckets
df['price_range'] = pd.cut(df['sale_price'], bins = [0,50,100,200,500,1000,
                                                     df['sale_price'].max()],
                                                     labels = ['<50','50-100','100-200','200-500','500-1000','1000+'])

In [ ]:
# 7. Save Cleaned Data
df.to_csv('../data/cleaned_bigbasket.csv', index = False)

In [ ]:
#  Clean DataFrame
df.head()

## Step 2 :- Feature Engineering
 ### 2A :- Extract Numeric Features from price_range
- extract numeric values from the `price_range` column, which contains price intervals as strings (e.g., `"200-500"`). The following features are engineered:

- **`min_price`**: The lower bound of the price range.
- **`max_price`**: The upper bound of the price range.
- **`mid_price`**: The average of `min_price` and `max_price`, representing a central estimate.

- These new features will help in price-based segmentation, trend analysis, and model training if used for predictions.

In [ ]:
df[['min_price', 'max_price']] = df['price_range'].str.extract(r'(\d+)-(\d+)').astype(float)
df['mid_price'] = (df['min_price'] + df['max_price']) / 2
df[['min_price', 'mid_price', 'max_price']].head() 

### 2B: Calculate Effective Price  
Using either of the following formulas:

- From absolute discount:  
  `effective_price = sale_price - discount`


This gives us the **actual price customers pay**, which allows more accurate price-based analysis.

## Step 3 :- Exploratory Data Analysis (EDA)
We'll explore:

- Category & Subcategory distribution

- Top brands

- Price analysis

- Discount distribution

- Ratings overview

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
# 1. Category-wise Product Count

# Top 10 categories
category_counts = df['category'].value_counts().nlargest(10)

# Create bar chart
plt.figure(figsize=(10,6))
category_counts.plot(kind='bar', color='skyblue')
plt.title('Top 10 Categories by Product Count')
plt.xlabel('Category')
plt.ylabel('Product Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save image
plt.savefig("../reports/category_distribution.png")
plt.show()


In [ ]:
# 2. Subcategory Distribution (within top Category)
top_cat = df['category'].value_counts().idxmax()
sub_df = df[df['category'] == top_cat]

fig = px.bar(sub_df['sub_category'].value_counts(),
             title = f'Subcategories under "{top_cat}"',
             labels = {'value' : 'Product Count', 'index' : 'Subcategory'})

fig.show()

In [ ]:
# 3. Top Brands
top_brands = df['brand'].value_counts().nlargest(10)

fig = px.bar(top_brands,
             title = 'Top 10 Brands by Product Listings',
             labels = {'value' : 'Product Count', 'index' : 'Brand'})

fig.show()

In [ ]:
# 4. Price vs. Discount %
fig = px.scatter(df, x='market_price', y='discount_percent',
                 color='category',
                 title='Discount % vs Market Price',
                 hover_data=['product','brand'])

fig.show()

In [ ]:
# 5. Price Range Distribution
fig = px.histogram(df, x='price_range', color='category',
                   title='Product Distribution by Price Range',
                   barmode='group')

fig.show()

In [ ]:
# 6. Ratings Distribution
plt.figure(figsize=(10,5))
sns.histplot(df['rating'], bins= 20, kde = True)
plt.title('Product Ratings Distribution')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

## Step 3: Insights & Suggestions

### A. Top Categories:
- Insight: Which categories have the highest number of products?
- Suggestion: Focus marketing efforts on categories with high product listings. You can create special promotions for these categories to boost sales.

In [ ]:
top_categories = df['category'].value_counts().nlargest(5)
top_categories

### B. Price Range Analysis:
- Insight: Products are spread across different price ranges. Where are the majority of products positioned?

- Suggestion: Offer discounts in price segments where many products exist to boost sales. Or create premium offerings in higher price ranges.

In [ ]:
price_range = df['sale_price'].describe()
price_range

### C. Discount Effectiveness:
- Insight: Are discounts effective across product categories? Do we see a correlation between discount and sales price?

- Suggestion: Products with higher discounts should be marketed aggressively to increase sales volume.

In [ ]:
discount_vs_price = df[['sale_price','market_price','discount_percent']].corr()
discount_vs_price

### D. Brand Performance:
- Insight: Which brands are the top performers?

- Suggestion: Partnerships or exclusive products with top-performing brands could enhance sales. You can also look at whether products from popular brands have higher ratings or sell faster.

In [ ]:
top_brands = df['brand'].value_counts().nlargest(5)
top_brands

### E. Rating Distribution:
- Insight: Which products are performing well based on user ratings? High ratings often correlate with customer loyalty.

- Suggestion: Promote products with higher ratings through targeted campaigns. Additionally, consider offering discounts for products with lower ratings to clear stock.

In [ ]:
high_rated_products = df[df['rating'] >= 4.0][['product', 'rating']]
high_rated_products.head(10)

### F. Suggestions Summary:
- Summarize all key insights in a text file for reporting:

In [ ]:
insights_text = f"""
📊 Business Insights Report

1. 🛒 **Top Product Categories**
   - The most stocked and diverse categories are:
     {top_categories}
   - 👉 *Recommendation:* Prioritize promotions and campaigns around these categories to attract more buyers.

2. 💸 **Average Product Pricing**
   - The average price for most products falls between:
     {price_range}
   - 👉 *Recommendation:* Offer special discounts on moderately priced products. This will help increase sales while keeping profit margins healthy.

3. 🧾 **Impact of Discounts on Sales**
   - There is a clear trend: **Products with higher discounts tend to sell more**. For example, products with discounts of **X%** show an increase in sales of **Y%**.
   - 👉 *Recommendation:* Focus on marketing products with attractive discounts, as these are likely to bring in more customers.

4. 🔝 **Top Performing Brands**
   - The most successful and popular brands are:
     {top_brands}
   - 👉 *Recommendation:* Strengthen relationships with these brands through collaborations, bundling, or exclusive deals.

5. ⭐ **Best Rated Products**
   - Products with **ratings above 4.0** are consistently preferred by customers.
   - 👉 *Recommendation:* Feature these highly-rated products prominently on your website and in promotional materials to boost customer confidence.

📝 **Summary**:
Focus your business strategies on high-performing categories and brands, offer targeted discounts on mid-priced items, and showcase top-rated products to improve customer trust and engagement.

"""


# Save the insights to a text file
with open("../reports/suggestion.txt", "w", encoding="utf-8") as f:
    f.write(insights_text)
